In [11]:
import pandas as pd
import os

# Charger le DataFrame (assurez-vous que le chemin est correct)
df = pd.read_csv("/Users/titou/Desktop/consolidation-etalab-schema-irve-statique-v-2.3.1-20250322.csv")

# 1️⃣ Vérification des valeurs manquantes (Complétude)
missing_values = df.isnull().sum()
missing_errors = missing_values[missing_values > 0]
print("\n🔍 Vérification des valeurs manquantes :")
print(missing_errors)

# Remplir les valeurs manquantes avec une valeur par défaut (exemple)
df.fillna({
    "nom_amenageur": "Inconnu",
    "siren_amenageur": "000000000",
    "contact_amenageur": "Inconnu",
    "nom_operateur": "Inconnu",
}, inplace=True)

# 2️⃣ Vérification des doublons sur les identifiants uniques (Cohérence)
unique_ids = ["id_station_itinerance", "id_pdc_itinerance", "num_pdl"]
duplicates_errors = {}
for uid in unique_ids:
    if uid in df.columns:
        duplicates = df[uid].duplicated().sum()
        duplicates_errors[uid] = duplicates
        if duplicates > 0:
            print(f"\n⚠️ {duplicates} doublons trouvés dans {uid}.")
            # Optionnel : supprimer les doublons
            df = df.drop_duplicates(subset=uid)

# 3️⃣ Vérification de l'exactitude des coordonnées GPS
df_invalid_gps = df[(df["consolidated_latitude"] < -90) | (df["consolidated_latitude"] > 90) |
                     (df["consolidated_longitude"] < -180) | (df["consolidated_longitude"] > 180)]
invalid_gps_errors = len(df_invalid_gps)
print(f"\n⚠️ {invalid_gps_errors} stations ont des coordonnées GPS incorrectes.")

# 4️⃣ Vérification des dates de mise à jour (Fréquence de mise à jour)
df_invalid_date = df[~df["date_maj"].astype(str).str.match(r'^\d{4}-\d{2}-\d{2}$', na=False)]
invalid_date_errors = len(df_invalid_date)
print(f"\n⚠️ {invalid_date_errors} enregistrements ont un format de date incorrect dans 'date_maj'.")

# 5️⃣ Vérification de la cohérence entre Code INSEE et Code Postal
df_insee_invalid = df[df["consolidated_is_code_insee_verified"] != "True"]
insee_invalid_errors = len(df_insee_invalid)
print(f"\n⚠️ {insee_invalid_errors} enregistrements avec Code INSEE non vérifié.")

# 6️⃣ Vérification de la puissance des bornes (Doit être > 0)
df_invalid_power = df[df["puissance_nominale"] <= 0]
invalid_power_errors = len(df_invalid_power)
print(f"\n⚠️ {len(df_invalid_power)} enregistrements avec une puissance nominale incorrecte.")

# 7️⃣ Vérification des modes de paiement
df_payment_missing = df[(df["paiement_acte"].isnull()) & (df["paiement_cb"].isnull()) & (df["paiement_autre"].isnull())]
payment_missing_errors = len(df_payment_missing)
print(f"\n⚠️ {len(df_payment_missing)} stations sans mode de paiement renseigné.")

# Vérification de la disponibilité des fichiers
file_path = "/Users/titou/Desktop/consolidation-etalab-schema-irve-statique-v-2.3.1-20250322.csv"
file_accessible = os.path.exists(file_path)

# Résumé des erreurs détectées
errors = {
    "Valeurs manquantes": missing_errors.sum(),
    "Doublons ID station": sum(duplicates_errors.values()),
    "SIREN incorrects": 0,  # Pas d'erreur spécifique sur SIREN ici mais peut être ajouté
    "Coordonnées GPS erronées": invalid_gps_errors,
    "Dates au mauvais format": invalid_date_errors,
    "Code INSEE non vérifié": insee_invalid_errors,
    "Puissance incorrecte": invalid_power_errors,
    "Stations sans paiement renseigné": payment_missing_errors,
    "Fichier accessible": "Oui" if file_accessible else "Non",
}

# Création du DataFrame pour le rapport
df_errors = pd.DataFrame(list(errors.items()), columns=["Type d'erreur", "Nombre d'occurrences"])

# Définir le chemin pour le rapport CSV
output_path = "/Users/titou/Desktop/rapport_verification.csv"  # Changez ce chemin selon vos permissions

# Sauvegarder le rapport CSV
df_errors.to_csv(output_path, index=False)

# Affichage du message de confirmation
print("\n🔍 **Résumé des erreurs détectées** 🔍")
for key, value in errors.items():
    print(f"{key}: {value} erreurs")

print(f"\n✅ Rapport généré : {output_path}")


/var/folders/tk/bwww5fgn60j227k3j1tklnm80000gn/T/ipykernel_76191/285350397.py:5: DtypeWarning: Columns (12,18,19,20,21,22,24,26,29,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/titou/Desktop/consolidation-etalab-schema-irve-statique-v-2.3.1-20250322.csv")



🔍 Vérification des valeurs manquantes :
nom_amenageur                1916
siren_amenageur             74076
contact_amenageur           62705
nom_operateur                3358
telephone_operateur         42654
id_station_local            45809
code_insee_commune          70367
id_pdc_local                47501
gratuit                       761
paiement_cb                 30752
paiement_autre              17494
tarification                98735
restriction_gabarit          2958
raccordement                63872
num_pdl                     91644
date_mise_en_service        38742
observations                86576
cable_t2_attache            54947
consolidated_code_postal    77604
consolidated_commune        70367
dtype: int64

⚠️ 75408 doublons trouvés dans id_station_itinerance.

⚠️ 2122 doublons trouvés dans id_pdc_itinerance.

⚠️ 44762 doublons trouvés dans num_pdl.

⚠️ 0 stations ont des coordonnées GPS incorrectes.

⚠️ 0 enregistrements ont un format de date incorrect dans 'date_maj